In [13]:
"""Machine learning functions"""

import logging
import joblib
import random
import pandas as pd

from fastapi import APIRouter
import pandas as pd
from pydantic import BaseModel, Field, validator
from sklearn.neighbors import NearestNeighbors



log = logging.getLogger(__name__)
router = APIRouter()

class User(BaseModel):
    """Use this data model to parse the request body JSON."""
    user_input: str = Field(..., example=
        """I am having trouble sleeping, I'm stressed and I don't want to eat. I would like to have an appetite, sleep well, and be able to relax.""")


    def to_df(self):
        """Convert pydantic object to pandas dataframe with 1 row."""
        return pd.DataFrame([dict(self)])


@router.post('/predict')
async def predict(user: User):
    """
    Make random baseline predictions for classification problem 🔮

    ### Request Body

    - `user_input:`: string

    ### Response
    - `strain_name`: string
    - `description`: string
    - `strain_effects`: string
    - `strain_ailments`: string
    """
    
    vectorizer = joblib.load('app/jl_tfidf.joblib')
    nn = joblib.load('app/jl_knn.joblib')

    MJ = pd.read_csv('data\MJ.csv')

    model_food = vectorizer.transform([user.user_input])

    strains = []
    for i in range(4):
        strain_name = (MJ.loc[nn.kneighbors(model_food.todense())[1][0][i]]['Strain']),
        description = (MJ.loc[nn.kneighbors(model_food.todense())[1][0][i]]['Description']),
        strain_effects = (MJ.loc[nn.kneighbors(model_food.todense())[1][0][i]]['Effects_y']),
        strain_ailments = (MJ.loc[nn.kneighbors(model_food.todense())[1][0][i]]['Ailment'])
        strains.append(strain_name, description, strain_effects, strain_ailments)
        return strains

    df = pd.DataFrame(strains, columns = ['name', 'description', 'effects', 'ailments'])
    result = df.to_json(orient="split")
    good_pot = json.dumps(result)
    return {
            good_pot
    }

In [14]:
a_user = User(user_input='icky, sore, lazy')

In [15]:
answer = predict(a_user)

In [16]:
str(answer)

'<coroutine object predict at 0x0000026755669140>'

In [29]:
strains = ['pot', 'good stuff', 'kite high', 'bored', 'npt', 'more stuff', 'chill', 'stressed']
strains

('pot',
 'good stuff',
 'kite high',
 'bored',
 'npt',
 'more stuff',
 'chill',
 'stressed')

In [31]:
# each sublist should have 4 elements 
n = 4
   
# using list comprehension  
strain_lists = [strains[i:i + n] for i in range(0, len(strains), n)]  
print(x) 

[('pot', 'good stuff', 'kite high', 'bored'), ('npt', 'more stuff', 'chill', 'stressed')]


In [37]:
    # df = pd.DataFrame(x, index = ['name', 'description', 'effects', 'ailments']).T
    df = pd.DataFrame(strains_list, columns = ['name', 'description', 'effects', 'ailments'])
    

,name,description,effects,ailments
0,pot,good stuff,kite high,bored
1,npt,more stuff,chill,stressed


In [40]:
result = df.to_json(orient="split", index=False)
result

'{"columns":["name","description","effects","ailments"],"data":[["pot","good stuff","kite high","bored"],["npt","more stuff","chill","stressed"]]}'

In [48]:
import json
# use list comprehension to break down strian info
n = 4
strain_list = [strains[i:i + n] for i in range(0, len(strains), n)]

strain_df = pd.DataFrame(strain_list, columns = ['name', 'description', 'effects', 'ailments'])
strain_json = strain_df.to_json(orient="split", index=False)
result = json.dumps(strain_json)

In [50]:
strain_json

'{"columns":["name","description","effects","ailments"],"data":[["pot","good stuff","kite high","bored"],["npt","more stuff","chill","stressed"]]}'

In [49]:
result

'"{\\"columns\\":[\\"name\\",\\"description\\",\\"effects\\",\\"ailments\\"],\\"data\\":[[\\"pot\\",\\"good stuff\\",\\"kite high\\",\\"bored\\"],[\\"npt\\",\\"more stuff\\",\\"chill\\",\\"stressed\\"]]}"'